# 1 Dependencies import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scripts.quickml.reader import read_csv
from scripts.quickml.plot import feature_distribution_univar, feature_filling, correlation_heatmap, feature_trend_bivar, feature_distribution_bivar, dataframe_distribution_overview
from scripts.quickml.compare import index_lth
from scripts.utils import fill_infos

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

np.random.seed(294697)

***
# 2 Data import

In [ ]:
data = pd.read_csv("datas/sample.csv", index_col=[0], sep=";")

In [ ]:
data.shape

***
# 3 Overview

In [ ]:
dataframe_distribution_overview(data)

***
# 4 Global correlations

In [ ]:
correlation_heatmap(data)

***
# 5 Nutriscore / Ecoscore correlations

In [ ]:
feature_trend_bivar(data, "nutriscore_score", "Nutriscore", "ecoscore_score_fr", "Ecoscore", 1000)

Nutriscore: the bigger the value, the worst the product is.  
Ecoscore: the bigger the value, the worst the environemental impact is.

## Conclusion:
The worst is a product (high nutriscore), the better it is for the environement (low ecoscore).

***
# 6 Nutriscore / Ecoscore distribution

In [ ]:
feature_distribution_bivar(data, "nutriscore_score", "Nutriscore", "ecoscore_score_fr", "Ecoscore", divider=10)

***
# 7 Nutriscore / countries disribution

In [ ]:
data.iloc[10:].countries_en.head(20)

The feature "countries_en" is in reality a list, as we can see at index 47266: France,United Kingdom.  
We will choose to keep only the last country in this list for each row.

In [ ]:
def keep_last_country(row):
    cotr = row["countries_en"]
    splt = cotr.split(",")
    row["countries_en"] = splt[-1]
    return row

data = data[data.countries_en.notna()].apply(lambda row: keep_last_country(row), axis=1)

In [ ]:
data.iloc[10:].countries_en.head(20)

Let's see now how the nutriscore is distributed by country.

In [ ]:
data_by_countries_nutriscore = data.groupby(["countries_en", "nutriscore_grade"]).nutriscore_grade.agg([len])
data_by_countries_nutriscore.head()

In [ ]:
data_by_countries_nutriscore = data_by_countries_nutriscore.reset_index()
data_by_countries_nutriscore.head()

In [ ]:
feature_distribution_univar(data_by_countries_nutriscore, "nutriscore_grade", "Nutriscore")

In [ ]:
nutriscore_distrib_per_country = data_by_countries_nutriscore.loc[data_by_countries_nutriscore.countries_en.isin(["France", "Germany", "Belgium", "Canad", "Italy", "Mexico", "Netherlands", "Poland", "Portugal", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States"])]

In [ ]:
plt.figure(figsize=(30,6))

sns.catplot(x="countries_en", y="len", hue="nutriscore_grade", data=nutriscore_distrib_per_country, kind="bar", height=5, aspect=5)

plt.title("Nutriscore grade distribution", size=20)
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel("Nutriscore grade", size=16)
plt.ylabel("Volume", size=16)
plt.show()

***
# 8 Ecoscore / countries distribution

Let's see now how the ecoscore is distributed by country.

In [ ]:
data_by_countries_ecoscore = data.groupby(["countries_en", "ecoscore_grade_fr"]).ecoscore_grade_fr.agg([len])
data_by_countries_ecoscore.head()

In [ ]:
data_by_countries_ecoscore = data_by_countries_ecoscore.reset_index()
data_by_countries_ecoscore.head()

In [ ]:
feature_distribution_univar(data_by_countries_ecoscore, "ecoscore_grade_fr", "Ecoscore")

In [ ]:
ecoscore_distrib_per_country = data_by_countries_ecoscore.loc[data_by_countries_ecoscore.countries_en.isin(["France", "Germany", "Belgium", "Canad", "Italy", "Mexico", "Netherlands", "Poland", "Portugal", "Spain", "Sweden", "Switzerland", "United Kingdom", "United States"])]

In [ ]:
plt.figure(figsize=(30,6))

sns.catplot(x="countries_en", y="len", hue="ecoscore_grade_fr", data=ecoscore_distrib_per_country, kind="bar", height=5, aspect=5)

plt.title("Nutriscore grade distribution", size=20)
plt.xticks(size=16)
plt.yticks(size=16)
plt.xlabel("Nutriscore grade", size=16)
plt.ylabel("Volume", size=16)
plt.show()

***
# ---

## categorical to numerical transformation test

In [ ]:
data["ecoscore_CAT"] = data["ecoscore_grade_fr"].astype("category").cat.codes

In [ ]:
data["nutriscore_CAT"] = data["nutriscore_grade"].astype("category").cat.codes

In [ ]:
data.loc[:, ["ecoscore_CAT", "nutriscore_CAT"]].head()

## correlation: nutriscore/ecoscore/countries

In [ ]:
data["countries_NUM"] = data["countries_en"].astype("category").cat.codes

In [ ]:
data.loc[:, ["countries_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["countries_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/brands

In [ ]:
data["brands_NUM"] = data["brands"].astype('category').cat.codes

In [ ]:
data.loc[:, ["brands_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["brands_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/packaging

In [ ]:
data["packaging_NUM"] = data["packaging"].astype("category").cat.codes

In [ ]:
data.loc[:, ["packaging_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["packaging_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/categories_en

In [ ]:
data["categories_en_NUM"] = data["categories_en"].astype("category").cat.codes

In [ ]:
data.loc[:, ["categories_en_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["categories_en_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/labels_en

In [ ]:
data["labels_en_NUM"] = data["labels_en"].astype("category").cat.codes

In [ ]:
data.loc[:, ["labels_en_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["labels_en_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/serving_quantity

In [ ]:
data["serving_quantity_NUM"] = data["serving_quantity"].astype("category").cat.codes

In [ ]:
data.loc[:, ["serving_quantity_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["serving_quantity_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/pnns_groups_1/pnns_groups_2

In [ ]:
data["pnns_groups_1_NUM"] = data["pnns_groups_1"].astype("category").cat.codes

In [ ]:
data["pnns_groups_2_NUM"] = data["pnns_groups_2"].astype("category").cat.codes

In [ ]:
data.loc[:, ["pnns_groups_1_NUM", "pnns_groups_2_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["pnns_groups_1_NUM", "pnns_groups_1_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/brand_owner

In [ ]:
data["brand_owner_NUM"] = data["brand_owner"].astype("category").cat.codes

In [ ]:
data.loc[:, ["brand_owner_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["brand_owner_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/nova_group

In [ ]:
data["nova_group_NUM"] = data["nova_group"].astype("category").cat.codes

In [ ]:
data.loc[:, ["nova_group_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["nova_group_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## correlation: nutriscore/ecoscore/additives_en

In [ ]:
data["additives_en_NUM"] = data["additives_en"].astype("category").cat.codes

In [ ]:
data.loc[:, ["additives_en_NUM", "nutriscore_CAT", "ecoscore_CAT"]].head()

In [ ]:
data.loc[:, ["additives_en_NUM", "nutriscore_CAT", "ecoscore_CAT"]].corr()

## corr recap

In [ ]:
correlation_heatmap(data)

boxplot

La définition des quantiles est correctement maîtrisées.

Les éventuelles valeurs aberrantes ont été mises en évidence. Une solution est proposée (elle doit être justifiée si plusieurs choix sont possibles).

Les distributions observées sont correctement caractérisées (uni, bi, multi-modale).

La définition mathématique de la corrélation et les hypothèses sous-jacentes ont été expliquées
